In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import math
import statsmodels.api as sm
from statsmodels import regression, stats
import statsmodels
import scipy

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import chisquare




from numpy.random import seed
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, silhouette_score



/Users/juliusraschke/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/juliusraschke/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
DATA_PATH = '/Users/juliusraschke/Documents/Quantitative Finance/Summer Semester 2/Advanced Quant Finance/Data'

### Load in VIX Term Structure

In [3]:
VIX_TS = pd.read_csv(DATA_PATH + '/VIX_TS_CUSTOM.csv')

VIX_TS = VIX_TS.rename({'Date':'DATE','VIX_1^2':'1 month','VIX_2^2':'2 month','VIX_3^2':'3 month','VIX_6^2':'6 month',
               'VIX_9^2':'9 month','VIX_12^2':'12 month'},axis=1)

VIX_TS['DATE'] = pd.to_datetime(VIX_TS['DATE'], format = '%d/%m/%Y')

### Load in VIX Future Returns

In [4]:
VIX_FUT = pd.read_csv(DATA_PATH + '/OUR_VIX_FUT_RET_ENTIRE.csv')
VIX_FUT = VIX_FUT.drop('Unnamed: 0',1)


<ipython-input-4-eca8a35d4dd3>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  VIX_FUT = VIX_FUT.drop('Unnamed: 0',1)


In [5]:

VIX_FUT = VIX_FUT.rename({'Date':'DATE','r_1^{VIX fut.}':'1 month','r_2^{VIX fut.}':'2 month','r_3^{VIX fut.}':'3 month',
                         'r_4^{VIX fut.}':'4 month','r_5^{VIX fut.}':'5 month','r_6^{VIX fut.}':'6 month'},axis=1)

In [6]:
column_mapping = {
    '1 month': 'V_F_R 1 month',
    '2 month': 'V_F_R 2 month',
    '3 month': 'V_F_R 3 month',
    '4 month': 'V_F_R 6 month',
    '5 month': 'V_F_R 9 month',
    '6 month': 'V_F_R 12 month',
}

VIX_FUT.rename(columns=column_mapping, inplace=True)

### Load in our Variance Swap Returns

In [7]:
VAR_SWAPS = pd.read_csv(DATA_PATH + '/VAR_SWAP_RET_CUSTOM.csv')
VAR_SWAPS = VAR_SWAPS.drop('Unnamed: 0',1)
VAR_SWAPS = VAR_SWAPS.dropna()

<ipython-input-7-52fd60118f46>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  VAR_SWAPS = VAR_SWAPS.drop('Unnamed: 0',1)


In [8]:
VAR_SWAPS = VAR_SWAPS.rename({'Date':'DATE','1m':'1 month','2m':'2 month','3m':'3 month',
                         '6m':'6 month','9m':'9 month','12m':'12 month'},axis=1)

In [9]:
column_mapping = {
    '1 month': 'V_S_R 1 month',
    '2 month': 'V_S_R 2 month',
    '3 month': 'V_S_R 3 month',
    '6 month': 'V_S_R 6 month',
    '9 month': 'V_S_R 9 month',
    '12 month': 'V_S_R 12 month',
}

VAR_SWAPS.rename(columns=column_mapping, inplace=True)

### Load in alternative Swap Returns

In [2477]:
VAR_SWAPS = pd.read_csv(DATA_PATH + '/VAR_SWAP_RET_BBG.csv')
VAR_SWAPS = VAR_SWAPS.drop('Unnamed: 0',1)
VAR_SWAPS = VAR_SWAPS.dropna()

<ipython-input-2477-0f0b29efc197>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  VAR_SWAPS = VAR_SWAPS.drop('Unnamed: 0',1)


In [2478]:
VAR_SWAPS = VAR_SWAPS.rename({'Date':'DATE','1m_BBG':'1 month','2m_BBG':'2 month','3m_BBG':'3 month',
                         '6m_BBG':'6 month','9m_BBG':'9 month','12m_BBG':'12 month'},axis=1)

In [2479]:
column_mapping = {
    '1 month': 'V_S_R 1 month',
    '2 month': 'V_S_R 2 month',
    '3 month': 'V_S_R 3 month',
    '6 month': 'V_S_R 6 month',
    '9 month': 'V_S_R 9 month',
    '12 month': 'V_S_R 12 month',
}

VAR_SWAPS.rename(columns=column_mapping, inplace=True)

In [2480]:
VAR_SWAPS = VAR_SWAPS[:2500]

### Load in Straddle Returns

In [10]:
STRADDLE = pd.read_csv(DATA_PATH + '/straddle_returns_correct.csv')

STRADDLE = STRADDLE.rename({'date':'DATE',
                            'return_30':'1 month','return_60':'2 month','return_90':'3 month',
                            'return_180':'6 month','return_270':'9 month','return_360':'12 month',
                            },axis=1)

STRADDLE_LONG = STRADDLE

In [11]:
column_mapping = {
    '1 month': 'S_R 1 month',
    '2 month': 'S_R 2 month',
    '3 month': 'S_R 3 month',
    '6 month': 'S_R 6 month',
    '9 month': 'S_R 9 month',
    '12 month': 'S_R 12 month',
}

STRADDLE_LONG.rename(columns=column_mapping, inplace=True)

### Get summary statistics for VIX_TS

In [12]:
VIX_TS.describe
VIX_TS.quantile(0.01)
VIX_TS.median()

<ipython-input-12-1a36c6aef8c0>:2: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  VIX_TS.quantile(0.01)
<ipython-input-12-1a36c6aef8c0>:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  VIX_TS.median()


1 month     0.031879
2 month     0.034022
3 month     0.035288
6 month     0.038493
9 month     0.040251
12 month    0.040942
dtype: float64

### VIX_TS PCA

In [13]:
pca1 = PCA()
X = VIX_TS.iloc[:,1:7]
pca1.fit(X)
print('Components:')
print(pca1.components_)
print('')
print('Variance Proportions:')
print(pca1.explained_variance_ratio_)
SLOPE1 = pca1.transform(X)[:,1]

pca = pca1.components_

Components:
[[ 0.53632223  0.47794522  0.43366676  0.34817237  0.30616177  0.28443091]
 [ 0.590623    0.21078547 -0.01793026 -0.33777433 -0.47101191 -0.52006609]
 [ 0.56227136 -0.39365363 -0.5703049  -0.10336025  0.18462831  0.39858387]
 [-0.20089877  0.6147487  -0.22228398 -0.59879547  0.00081283  0.41684156]
 [-0.08192512  0.44069467 -0.66097825  0.53212865  0.04451822 -0.27756343]
 [ 0.01737258  0.00684912 -0.00784512 -0.33508842  0.80519725 -0.4888383 ]]

Variance Proportions:
[9.64019017e-01 3.42489252e-02 1.39286802e-03 2.12802987e-04
 1.02710122e-04 2.36768828e-05]


In [14]:
X_trans = pca1.fit_transform(X)
X_trans = pd.DataFrame(data=X_trans)
X_trans['Date']=VIX_TS['DATE']
X_trans = X_trans[['Date',0,1,2,3,4,5]]

for value in X_trans.var():
    print(value*100000)

643.8368378861719
22.873739358863247
0.9302510911458216
0.1421241697927066
0.06859673794379725
0.015813017281843488


<ipython-input-14-26b6029a49a1>:6: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  for value in X_trans.var():


### Load in Interest Rate Returns

In [15]:
df = pd.read_csv(DATA_PATH + '/IR.csv')

In [16]:
# Get rates for all days which are in the 300s
filtered_df = df[(df['days'] >= 300) & (df['days'] < 400)].reset_index(drop=True)

# Sort the DataFrame by 'dates' and 'days'
df_sorted = filtered_df.sort_values(by=['date', 'days'])

# Keep the rows with the closest 'days' value to 365 for each date
IR_TS = df_sorted.groupby('date', group_keys=False).apply(lambda group: group.iloc[(group['days'] - 365).abs().argsort()[:1]])

# Reset the index if needed
IR_TS.reset_index(drop=True, inplace=True)

In [17]:
IR_TS['Daily Rate'] = ((1+IR_TS['rate'])**(1/IR_TS['days']))-1
IR_TS = IR_TS.drop(['rate','days'],axis=1)

In [18]:
IR_TS.rename(columns={'date': 'DATE'}, inplace=True)

### Created Merged Databases for returns

In [19]:
IR_TS.set_index('DATE', inplace=True)
VIX_TS.set_index('DATE', inplace=True)
VAR_SWAPS.set_index('DATE', inplace=True)
VIX_FUT.set_index('DATE', inplace=True)
STRADDLE_LONG.set_index('DATE', inplace=True)

In [20]:
IR_TS.index = pd.to_datetime(IR_TS.index)
VIX_TS.index = pd.to_datetime(VIX_TS.index)
VAR_SWAPS.index = pd.to_datetime(VAR_SWAPS.index)
VIX_FUT.index = pd.to_datetime(VIX_FUT.index)
STRADDLE_LONG.index = pd.to_datetime(STRADDLE_LONG.index)

In [21]:
merged = pd.merge(VIX_TS,IR_TS, how='inner', left_index=True, right_index=True)

In [22]:
VAR_SWAPS = VAR_SWAPS/100
VIX_FUT = VIX_FUT/100
STRADDLE_LONG = STRADDLE_LONG

In [23]:
VAR_SWAPS_MERGED=pd.merge(merged,VAR_SWAPS, how='inner', left_index=True, right_index=True)
VIX_FUT_MERGED=pd.merge(merged,VIX_FUT, how='inner', left_index=True, right_index=True)
STRADDLE_MERGED=pd.merge(merged,STRADDLE_LONG, how='inner', left_index=True, right_index=True)

### Calculate Excess returns

In [24]:
XS_VAR_SWAPS = VAR_SWAPS_MERGED.iloc[:, 7:]
columns_to_subtract_from = ['V_S_R 1 month','V_S_R 2 month', 'V_S_R 3 month', 'V_S_R 6 month', 'V_S_R 9 month', 'V_S_R 12 month']

# Iterate through the columns and subtract 'column_to_subtract' from each of them
for col in columns_to_subtract_from:
    XS_VAR_SWAPS[col] = XS_VAR_SWAPS[col] - VAR_SWAPS_MERGED['Daily Rate']

In [25]:
XS_VIX_FUT = VIX_FUT_MERGED.iloc[:, 7:]
columns_to_subtract_from = ['V_F_R 1 month','V_F_R 2 month', 'V_F_R 3 month', 'V_F_R 6 month', 'V_F_R 9 month', 'V_F_R 12 month']

# Iterate through the columns and subtract 'column_to_subtract' from each of them
for col in columns_to_subtract_from:
    XS_VIX_FUT[col] = XS_VIX_FUT[col] - VIX_FUT_MERGED['Daily Rate']

In [26]:
XS_STRADDLE = STRADDLE_MERGED.iloc[:, 7:]
columns_to_subtract_from = ['S_R 1 month','S_R 2 month', 'S_R 3 month', 'S_R 6 month', 'S_R 9 month', 'S_R 12 month']

# Iterate through the columns and subtract 'column_to_subtract' from each of them
for col in columns_to_subtract_from:
    XS_STRADDLE[col] = XS_STRADDLE[col] - STRADDLE_MERGED['Daily Rate']

### Variance Assets Excess return summary

In [27]:
XS_VAR_SWAPS.describe()
XS_VAR_SWAPS.skew()

V_S_R 1 month     2.899152
V_S_R 2 month     2.330483
V_S_R 3 month     2.037821
V_S_R 6 month     1.907296
V_S_R 9 month     1.772701
V_S_R 12 month    1.734289
dtype: float64

In [28]:
XS_VIX_FUT.describe()
XS_VIX_FUT.skew()

V_F_R 1 month    -0.135308
V_F_R 2 month    -0.861374
V_F_R 3 month    -4.473292
V_F_R 6 month    -7.132690
V_F_R 9 month    -8.187459
V_F_R 12 month   -7.713339
dtype: float64

In [29]:
XS_STRADDLE.describe()
XS_STRADDLE.skew()

S_R 1 month     3.465697
S_R 2 month     3.087170
S_R 3 month     2.673082
S_R 6 month     2.614973
S_R 9 month     2.308061
S_R 12 month    1.941775
dtype: float64

### Expectation Hypothesis Test

### Version 1

In [30]:
VIX_TS.reset_index(drop=True, inplace=True)

In [31]:
n_days = 21

In [32]:
Y_1 = VIX_TS['1 month'][n_days:].reset_index(drop=True) - VIX_TS['2 month'][:-n_days]
Y_2 = VIX_TS['2 month'][n_days:].reset_index(drop=True) - VIX_TS['3 month'][:-n_days]
Y_3 = VIX_TS['3 month'][3*n_days:].reset_index(drop=True) - VIX_TS['6 month'][:-3*n_days]
Y_4 = VIX_TS['6 month'][3*n_days:].reset_index(drop=True) - VIX_TS['9 month'][:-3*n_days]
Y_5 = VIX_TS['9 month'][3*n_days:].reset_index(drop=True) - VIX_TS['12 month'][:-3*n_days]

X_1 = (VIX_TS['2 month'][:-n_days] - VIX_TS['1 month'][:-n_days]).reset_index(drop=True)
X_2 = ((1/2)*VIX_TS['3 month'][:-n_days] - VIX_TS['1 month'][:-n_days]).reset_index(drop=True)
X_3 = (VIX_TS['6 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days]).reset_index(drop=True)
X_4 = ((1/2)*VIX_TS['9 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days]).reset_index(drop=True)
X_5 = ((1/3)*VIX_TS['12 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days]).reset_index(drop=True)

In [33]:
data_1 = {'b_1': X_1,
        'b_2': X_2,
        'c_1': VIX_TS['1 month'][:-n_days].reset_index(drop=True),
       'c_2': VIX_TS['2 month'][:-n_days].reset_index(drop=True)}

data_2 = {'b_3': X_3,
        'b_4': X_4,
        'b_5': X_5,
       'c_3': VIX_TS['3 month'][:-3*n_days].reset_index(drop=True),
       'c_4': VIX_TS['6 month'][:-3*n_days].reset_index(drop=True),
       'c_5': VIX_TS['9 month'][:-3*n_days].reset_index(drop=True),}

df_1 = pd.DataFrame(data_1)
df_2 = pd.DataFrame(data_2)

In [34]:
X = df_2[['b_5','c_5']]
Y = Y_5

X = sm.add_constant(X)  # Add a constant (intercept) to the independent variables
model = sm.OLS(Y, X).fit()

# Calculate Newey West

cov_mat = stats.sandwich_covariance.cov_hac(model)
newey_west_se = np.sqrt(np.diag(cov_mat))

# Get the regression coefficients, including the intercept
coefficients = model.params

# Calculate p-values for the expectation hypothesis (b=1)
t_values = (coefficients[1]-1) / newey_west_se[1]

#Calculate p-values
p_values = 2 * scipy.stats.t.sf(abs(t_values), model.df_resid)

# Calculate R-squared
r_squared = model.rsquared

# Output the results
print("Regression coefficients (including intercept):", coefficients)
print("Newey-West standard errors:", newey_west_se)
print("P-values for the expectation hypothesis (b=1):", p_values.round(3))
print("R-squared:", r_squared)

Regression coefficients (including intercept): const    0.014353
b_5     -0.083862
c_5     -0.370879
dtype: float64
Newey-West standard errors: [0.00151606 0.09035461 0.08215647]
P-values for the expectation hypothesis (b=1): 0.0
R-squared: 0.13133920172422064


/Users/juliusraschke/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


### Version 2

In [35]:
Y_1 = VIX_TS['1 month'][n_days:].reset_index(drop=True) - VIX_TS['1 month'][:-n_days]
Y_2 = VIX_TS['2 month'][n_days:].reset_index(drop=True) - VIX_TS['2 month'][:-n_days]
Y_3 = VIX_TS['3 month'][3*n_days:].reset_index(drop=True) - VIX_TS['3 month'][:-3*n_days]
Y_4 = VIX_TS['6 month'][3*n_days:].reset_index(drop=True) - VIX_TS['6 month'][:-3*n_days]
Y_5 = VIX_TS['9 month'][3*n_days:].reset_index(drop=True) - VIX_TS['9 month'][:-3*n_days]

X_1 = (VIX_TS['2 month'][:-n_days] - VIX_TS['1 month'][:-n_days])+(VIX_TS['2 month'][:-n_days] - VIX_TS['1 month'][:-n_days])
X_2 = (VIX_TS['3 month'][:-n_days] - VIX_TS['2 month'][:-n_days])+((1/2)*VIX_TS['3 month'][:-n_days] - VIX_TS['1 month'][:-n_days])
X_3 = (VIX_TS['6 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days])+(VIX_TS['6 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days])
X_4 = (VIX_TS['9 month'][:-3*n_days] - VIX_TS['6 month'][:-3*n_days])+((1/2)*VIX_TS['9 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days])
X_5 = (VIX_TS['12 month'][:-3*n_days] - VIX_TS['9 month'][:-3*n_days])+((1/3)*VIX_TS['12 month'][:-3*n_days] - VIX_TS['3 month'][:-3*n_days])

In [36]:
data_1 = {'b_1': X_1,
        'b_2': X_2,
        'c_1': VIX_TS['1 month'][:-n_days].reset_index(drop=True),
       'c_2': VIX_TS['2 month'][:-n_days].reset_index(drop=True)}

data_2 = {'b_3': X_3,
        'b_4': X_4,
        'b_5': X_5,
       'c_3': VIX_TS['3 month'][:-3*n_days].reset_index(drop=True),
       'c_4': VIX_TS['6 month'][:-3*n_days].reset_index(drop=True),
       'c_5': VIX_TS['9 month'][:-3*n_days].reset_index(drop=True),}

df_1 = pd.DataFrame(data_1)
df_2 = pd.DataFrame(data_2)

In [37]:
X = df_2[['b_5','c_5']]
Y = Y_5

X = sm.add_constant(X)  # Add a constant (intercept) to the independent variables

model = sm.OLS(Y, X).fit()

# Calculate Newey West

cov_mat = stats.sandwich_covariance.cov_hac(model)
newey_west_se = np.sqrt(np.diag(cov_mat))

# Get the regression coefficients, including the intercept
coefficients = model.params

# Calculate p-values for the expectation hypothesis (b=1)
t_values = (coefficients[1]-1) / newey_west_se[1]

#Calculate p-values
p_values = 2 * scipy.stats.t.sf(abs(t_values), model.df_resid)

# Calculate R-squared
r_squared = model.rsquared

# Output the results
print("Regression coefficients (including intercept):", coefficients)
print("Newey-West standard errors:", newey_west_se)
print("P-values for the expectation hypothesis (b=1):", p_values.round(3))
print("R-squared:", r_squared)

Regression coefficients (including intercept): const    0.014685
b_5      0.060630
c_5     -0.280989
dtype: float64
Newey-West standard errors: [0.00155818 0.08102579 0.07730431]
P-values for the expectation hypothesis (b=1): 0.0
R-squared: 0.17567405125965097


/Users/juliusraschke/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


### Converting daily to monthly data

In [38]:
cr_VAR_SWAPS = ((XS_VAR_SWAPS)+1).rolling(n_days).apply(np.prod)-1
cr_VAR_SWAPS[:-(n_days-1):] = cr_VAR_SWAPS[(n_days-1)::]
cr_VAR_SWAPS = cr_VAR_SWAPS[:-(n_days-1):]

cr_VIX_FUT = ((XS_VIX_FUT)+1).rolling(n_days).apply(np.prod)-1
cr_VIX_FUT[:-(n_days-1):] = cr_VIX_FUT[(n_days-1)::]
cr_VIX_FUT = cr_VIX_FUT[:-(n_days-1):]

cr_STRADDLE = ((XS_STRADDLE)+1).rolling(n_days).apply(np.prod)-1
cr_STRADDLE[:-(n_days-1):] = cr_STRADDLE[(n_days-1)::]
cr_STRADDLE = cr_STRADDLE[:-(n_days-1):]

In [39]:
cr_VAR_SWAPS.describe()
cr_VAR_SWAPS.skew()

V_S_R 1 month     16.530134
V_S_R 2 month     16.932126
V_S_R 3 month     15.745831
V_S_R 6 month     13.235393
V_S_R 9 month     10.688207
V_S_R 12 month     9.624279
dtype: float64

In [40]:
cr_VIX_FUT.describe()
cr_VIX_FUT.skew()

V_F_R 1 month     4.491419
V_F_R 2 month     3.617976
V_F_R 3 month     2.425840
V_F_R 6 month     1.026697
V_F_R 9 month    -0.259983
V_F_R 12 month   -1.039753
dtype: float64

In [41]:
cr_STRADDLE.describe()
cr_STRADDLE.skew()

S_R 1 month     5.509445
S_R 2 month     6.189848
S_R 3 month     5.014605
S_R 6 month     4.393542
S_R 9 month     3.427591
S_R 12 month    2.856287
dtype: float64

### Single Factor Test

In [42]:
pca1 = PCA()
scaler = StandardScaler()

### S&P 500 variance Swap Returns

### PCA

In [43]:
X = VAR_SWAPS_MERGED.iloc[:,:6]
X_trans = pca1.fit_transform(X)
X_trans = pd.DataFrame(data=X_trans)
scaled = scaler.fit_transform(X_trans)
scaled_VS = pd.DataFrame(data=scaled)
scaled_VS[['Level','Slope','Curve','PC4','PC5','PC6']] = scaled_VS[[0,1,2,3,4,5]]
scaled_VS = scaled_VS.drop([0,1,2,3,4,5],axis=1)
scaled_VS['Slope'] = -scaled_VS['Slope']

### Next day

In [44]:
XS_V_S = XS_VAR_SWAPS
XS_V_S.reset_index(drop=False, inplace=True)

In [45]:
X = scaled_VS[:-1]
Y = (XS_V_S['V_S_R 1 month'][1:]*10000).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':0})

In [46]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          V_S_R 1 month   R-squared (uncentered):                   0.006
Model:                            OLS   Adj. R-squared (uncentered):              0.005
Method:                 Least Squares   F-statistic:                              4.313
Date:                Sat, 02 Dec 2023   Prob (F-statistic):                    0.000239
Time:                        15:17:59   Log-Likelihood:                         -59493.
No. Observations:                6819   AIC:                                  1.190e+05
Df Residuals:                    6813   BIC:                                  1.190e+05
Df Model:                           6                                                  
Covariance Type:                  HAC                                                  
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Level        -87.4205     22.371     -3.908      0.000    -131.266     -43.575
Slope         25.2123     23.673      1.065      0.287     -21.186      71.611
Curve        -42.5654     21.570     -1.973      0.048     -84.842      -0.289
PC4           59.4976     21.018      2.831      0.005      18.303     100.692
PC5           -8.4924     21.510     -0.395      0.693     -50.651      33.666
PC6          -10.2438     20.155     -0.508      0.611     -49.746      29.259
==============================================================================
Omnibus:                     5027.067   Durbin-Watson:                   2.111
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           284803.658
Skew:                           2.975   Prob(JB):                         0.00
Kurtosis:                      34.097   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 0 lags and without small sample correction
"""

In [47]:
X_restricted = X[['Slope']]

# Fit the restricted model
model_restricted = sm.OLS(Y, X_restricted).fit(cov_type='HAC', cov_kwds={'maxlags': 0})

# Perform the F-test (Chi-square test)
f_test = model.compare_f_test(model_restricted)

print(f"F-statistic: {f_test[0]}, P-value: {f_test[1]}")

F-statistic: 8.099058694915644, P-value: 1.249232868481309e-07


/Users/juliusraschke/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:2167: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  warnings.warn('F test for comparison is likely invalid with ' +


### Next month

In [48]:
cr_V_S = cr_VAR_SWAPS
cr_V_S.reset_index(drop=False, inplace=True)

In [49]:
X = scaled_VS[:-n_days]
Y = (cr_V_S['V_S_R 1 month'][1:]*100).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':n_days})

In [50]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          V_S_R 1 month   R-squared (uncentered):                   0.024
Model:                            OLS   Adj. R-squared (uncentered):              0.023
Method:                 Least Squares   F-statistic:                              6.406
Date:                Sat, 02 Dec 2023   Prob (F-statistic):                    9.64e-07
Time:                        15:17:59   Log-Likelihood:                         -41348.
No. Observations:                6799   AIC:                                  8.271e+04
Df Residuals:                    6793   BIC:                                  8.275e+04
Df Model:                           6                                                  
Covariance Type:                  HAC                                                  
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Level        -13.5504      3.322     -4.079      0.000     -20.061      -7.040
Slope         -1.6349      3.319     -0.493      0.622      -8.140       4.870
Curve          0.6119      2.742      0.223      0.823      -4.762       5.985
PC4            7.3609      2.113      3.483      0.000       3.219      11.503
PC5           -3.7670      1.874     -2.010      0.044      -7.440      -0.094
PC6            4.0456      3.676      1.100      0.271      -3.160      11.251
==============================================================================
Omnibus:                    13812.681   Durbin-Watson:                   0.183
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         48562107.470
Skew:                          16.774   Prob(JB):                         0.00
Kurtosis:                     415.669   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 21 lags and without small sample correction
"""

### VIX Futures Returns

### PCA

In [51]:
X = VIX_FUT_MERGED.iloc[:,:6]
X_trans = pca1.fit_transform(X)
X_trans = pd.DataFrame(data=X_trans)
scaled = scaler.fit_transform(X_trans)
scaled_VF = pd.DataFrame(data=scaled)
scaled_VF[['Level','Slope','Curve','PC4','PC5','PC6']] = scaled_VF[[0,1,2,3,4,5]]
scaled_VF = scaled_VF.drop([0,1,2,3,4,5],axis=1)
scaled_VF['Slope'] = -scaled_VF['Slope']

### Next day

In [52]:
XS_V_F = XS_VIX_FUT

In [53]:
X = scaled_VF[:-1]
Y = (XS_V_F['V_F_R 12 month'][1:]*10000).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':0})

# Get the names of all parameters
params = model.params.index

# Create an R matrix for the Wald test
# Exclude 'const' and 'Curve' from the hypothesis
R = np.zeros((len(params)-1, len(params)))
row = 0
for i, param in enumerate(params):
    if param not in ['Slope']:
        R[row, i] = 1
        row += 1

# Perform the Wald Test (Chi-square test)
wald_test = model.wald_test(R)

print(f"Chi-squared statistic: {wald_test.statistic}, P-value: {wald_test.pvalue}")

Chi-squared statistic: [[0.74981978]], P-value: 0.980122989713167


In [54]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:         V_F_R 12 month   R-squared (uncentered):                   0.001
Model:                            OLS   Adj. R-squared (uncentered):             -0.000
Method:                 Least Squares   F-statistic:                             0.4861
Date:                Sat, 02 Dec 2023   Prob (F-statistic):                       0.819
Time:                        15:18:07   Log-Likelihood:                         -33165.
No. Observations:                4750   AIC:                                  6.634e+04
Df Residuals:                    4744   BIC:                                  6.638e+04
Df Model:                           6                                                  
Covariance Type:                  HAC                                                  
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Level         -0.1737      5.481     -0.032      0.975     -10.916      10.568
Slope         -5.9570      6.387     -0.933      0.351     -18.475       6.561
Curve         -0.0991      5.680     -0.017      0.986     -11.232      11.034
PC4            2.4104      4.656      0.518      0.605      -6.715      11.535
PC5            3.9093      5.750      0.680      0.497      -7.361      15.179
PC6            0.2039      3.875      0.053      0.958      -7.391       7.799
==============================================================================
Omnibus:                     6869.913   Durbin-Watson:                   2.270
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         19419084.989
Skew:                          -7.746   Prob(JB):                         0.00
Kurtosis:                     315.854   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 0 lags and without small sample correction
"""

### Next month

In [55]:
cr_V_F = cr_VIX_FUT

In [56]:
X = scaled_VF[:-n_days]
Y = (cr_V_F['V_F_R 12 month'][1:]*100).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':n_days})

# Get the names of all parameters
params = model.params.index

# Create an R matrix for the Wald test
# Exclude 'const' and 'Curve' from the hypothesis
R = np.zeros((len(params)-1, len(params)))
row = 0
for i, param in enumerate(params):
    if param not in ['Slope']:
        R[row, i] = 1
        row += 1

# Perform the Wald Test (Chi-square test)
wald_test = model.wald_test(R)

print(f"Chi-squared statistic: {wald_test.statistic}, P-value: {wald_test.pvalue}")

Chi-squared statistic: [[13.67462221]], P-value: 0.01781386006236348


In [57]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:         V_F_R 12 month   R-squared (uncentered):                   0.039
Model:                            OLS   Adj. R-squared (uncentered):              0.038
Method:                 Least Squares   F-statistic:                              3.855
Date:                Sat, 02 Dec 2023   Prob (F-statistic):                    0.000769
Time:                        15:18:23   Log-Likelihood:                         -17857.
No. Observations:                4730   AIC:                                  3.573e+04
Df Residuals:                    4724   BIC:                                  3.576e+04
Df Model:                           6                                                  
Covariance Type:                  HAC                                                  
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Level          0.0667      0.490      0.136      0.892      -0.894       1.027
Slope         -1.7616      0.577     -3.052      0.002      -2.893      -0.630
Curve          0.7041      0.432      1.630      0.103      -0.143       1.551
PC4            0.4317      0.368      1.172      0.241      -0.290       1.153
PC5            0.8557      0.331      2.584      0.010       0.207       1.505
PC6           -0.0647      0.475     -0.136      0.892      -0.995       0.865
==============================================================================
Omnibus:                     2325.733   Durbin-Watson:                   0.143
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           194182.939
Skew:                          -1.453   Prob(JB):                         0.00
Kurtosis:                      34.254   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 21 lags and without small sample correction
"""

### Straddle Returns

### PCA

In [58]:
X = STRADDLE_MERGED.iloc[:,:6]
X_trans = pca1.fit_transform(X)
X_trans = pd.DataFrame(data=X_trans)
scaled = scaler.fit_transform(X_trans)
scaled_S = pd.DataFrame(data=scaled)
scaled_S[['Level','Slope','Curve','PC4','PC5','PC6']] = scaled_S[[0,1,2,3,4,5]]
scaled_S = scaled_S.drop([0,1,2,3,4,5],axis=1)
scaled_S['Slope'] = -scaled_S['Slope']

### Next day

In [59]:
XS_S = XS_STRADDLE

In [60]:
X = scaled_S[:-1]
Y = (XS_S['S_R 12 month'][1:]*10000).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':0})

# Get the names of all parameters
params = model.params.index

# Create an R matrix for the Wald test
# Exclude 'const' and 'Curve' from the hypothesis
R = np.zeros((len(params)-1, len(params)))
row = 0
for i, param in enumerate(params):
    if param not in ['Slope']:
        R[row, i] = 1
        row += 1

# Perform the Wald Test (Chi-square test)
wald_test = model.wald_test(R)

print(f"Chi-squared statistic: {wald_test.statistic}, P-value: {wald_test.pvalue}")

Chi-squared statistic: [[3.77365575]], P-value: 0.582442049715624


In [61]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           S_R 12 month   R-squared (uncentered):                   0.006
Model:                            OLS   Adj. R-squared (uncentered):              0.005
Method:                 Least Squares   F-statistic:                              4.520
Date:                Sat, 02 Dec 2023   Prob (F-statistic):                    0.000140
Time:                        15:18:25   Log-Likelihood:                         -44397.
No. Observations:                6815   AIC:                                  8.881e+04
Df Residuals:                    6809   BIC:                                  8.885e+04
Df Model:                           6                                                  
Covariance Type:                  HAC                                                  
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Level         -0.1342      4.029     -0.033      0.973      -8.031       7.762
Slope        -12.0341      4.133     -2.912      0.004     -20.134      -3.934
Curve         -1.6847      3.823     -0.441      0.659      -9.177       5.808
PC4           -3.7700      3.082     -1.223      0.221      -9.811       2.271
PC5           -1.7959      3.770     -0.476      0.634      -9.184       5.592
PC6            1.8901      2.907      0.650      0.516      -3.807       7.588
==============================================================================
Omnibus:                     3412.026   Durbin-Watson:                   1.950
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            98374.583
Skew:                           1.818   Prob(JB):                         0.00
Kurtosis:                      21.254   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 0 lags and without small sample correction
"""

### Next month

In [62]:
cr_S = cr_STRADDLE
cr_S.reset_index(drop=False, inplace=True)

In [63]:
X = scaled_S[:-n_days]
Y = (cr_S['S_R 12 month'][1:]*100).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':n_days})

# Get the names of all parameters
params = model.params.index

# Create an R matrix for the Wald test
# Exclude 'const' and 'Curve' from the hypothesis
R = np.zeros((len(params)-1, len(params)))
row = 0
for i, param in enumerate(params):
    if param not in ['Slope']:
        R[row, i] = 1
        row += 1

# Perform the Wald Test (Chi-square test)
wald_test = model.wald_test(R)

print(f"Chi-squared statistic: {wald_test.statistic}, P-value: {wald_test.pvalue}")

Chi-squared statistic: [[3.57213216]], P-value: 0.6125023237791027


In [64]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           S_R 12 month   R-squared (uncentered):                   0.078
Model:                            OLS   Adj. R-squared (uncentered):              0.077
Method:                 Least Squares   F-statistic:                              8.390
Date:                Sat, 02 Dec 2023   Prob (F-statistic):                    4.34e-09
Time:                        15:18:26   Log-Likelihood:                         -24813.
No. Observations:                6795   AIC:                                  4.964e+04
Df Residuals:                    6789   BIC:                                  4.968e+04
Df Model:                           6                                                  
Covariance Type:                  HAC                                                  
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Level         -0.0955      0.432     -0.221      0.825      -0.943       0.752
Slope         -2.5949      0.433     -5.987      0.000      -3.444      -1.745
Curve          0.0721      0.394      0.183      0.855      -0.700       0.844
PC4           -0.2934      0.385     -0.761      0.447      -1.049       0.462
PC5           -0.2250      0.319     -0.705      0.481      -0.851       0.401
PC6            0.6467      0.442      1.463      0.143      -0.219       1.513
==============================================================================
Omnibus:                     4588.879   Durbin-Watson:                   0.094
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           134741.047
Skew:                           2.816   Prob(JB):                         0.00
Kurtosis:                      24.076   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 21 lags and without small sample correction
"""

### Economic Significance of Slope as a predictor

In [65]:
def std_error_range(data):
    a = data.iloc[:1,:]**2
    a = a.reset_index(drop=True)
    b = data.iloc[4:,:]**2
    b = b.reset_index(drop=True)
    return np.sqrt(a+b)

In [66]:
ES_VS = XS_VAR_SWAPS[1:].reset_index(drop=True)
ES_VS['Slope_Quantile'] = pd.qcut(scaled_VS['Slope'], q=5, labels=False)[:-1]
VS_Q_Ret = ES_VS.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(VS_Q_Ret.min() - VS_Q_Ret.max()).T
range_row.index = ['Range']
VS_Q_Ret = pd.concat([VS_Q_Ret, range_row])
VS_Q_Ret

,V_S_R 1 month,V_S_R 2 month,V_S_R 3 month,V_S_R 6 month,V_S_R 9 month,V_S_R 12 month
0,0.143824,0.254507,0.251221,0.207801,0.185636,0.164947
1,1.561576,0.918996,0.716891,0.445428,0.318274,0.261375
2,1.038921,0.516757,0.369068,0.153290,0.092315,0.075497
3,1.168157,0.700646,0.498613,0.237713,0.169856,0.139922
4,0.860468,0.275717,0.079422,-0.064377,-0.113321,-0.127512
Range,-1.417752,-0.664489,-0.637470,-0.509804,-0.431595,-0.388887


In [67]:
ES_VF = XS_VIX_FUT[1:].reset_index(drop=True,inplace=False)
ES_VF['Slope_Quantile'] = pd.qcut(scaled_VF['Slope'], q=5, labels=False)[:-1]
grouped = ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile')
mean = grouped.mean(numeric_only=True) * 100
std_dev = grouped.std(numeric_only=True)
count = grouped.count()
std_error = std_dev / np.sqrt(count)*100
std_error = std_error[['V_F_R 1 month','V_F_R 2 month','V_F_R 3 month','V_F_R 6 month','V_F_R 9 month','V_F_R 12 month']]
# Add a suffix to distinguish between mean and standard error columns
mean = mean.add_suffix('_Mean')
std_error = std_error.add_suffix('_StdErr')

# Concatenate mean and standard error DataFrames
result = pd.concat([mean, std_error], axis=1)

# Calculate the range row and add to the result
range_row = pd.DataFrame(result.min() - result.max()).T
range_row.index = ['Range']
result = pd.concat([result, range_row])
result

std_error_range(std_error)


,V_F_R 1 month_StdErr,V_F_R 2 month_StdErr,V_F_R 3 month_StdErr,V_F_R 6 month_StdErr,V_F_R 9 month_StdErr,V_F_R 12 month_StdErr
0,0.236179,0.196236,0.165679,0.1526,0.147937,0.149401


In [68]:
ES_STRADDLE =XS_STRADDLE[1:].reset_index(drop=True)
ES_STRADDLE['Slope_Quantile'] = pd.qcut(scaled_S['Slope'], q=5, labels=False)[:-1]

grouped = ES_STRADDLE.sort_values(by='Slope_Quantile').groupby('Slope_Quantile')
mean = grouped.mean(numeric_only=True) * 100
std_dev = grouped.std(numeric_only=True)
count = grouped.count()
std_error = std_dev / np.sqrt(count)*100
std_error = std_error[['S_R 1 month','S_R 2 month','S_R 3 month','S_R 6 month','S_R 9 month','S_R 12 month']]
# Add a suffix to distinguish between mean and standard error columns
mean = mean.add_suffix('_Mean')
std_error = std_error.add_suffix('_StdErr')

# Concatenate mean and standard error DataFrames
result = pd.concat([mean, std_error], axis=1)

# Calculate the range row and add to the result
range_row = pd.DataFrame(result.min() - result.max()).T
range_row.index = ['Range']
result = pd.concat([result, range_row])
result

std_error_range(std_error)

,S_R 1 month_StdErr,S_R 2 month_StdErr,S_R 3 month_StdErr,S_R 6 month_StdErr,S_R 9 month_StdErr,S_R 12 month_StdErr
0,0.253823,0.162199,0.130574,0.093988,0.076734,0.072886


### Removing Crisis

In [69]:
CRASH_VS_I_gfc = [VAR_SWAPS_MERGED.index.get_loc('2008-01-02'),VAR_SWAPS_MERGED.index.get_loc('2009-12-31')]
CRASH_VF_I_gfc = [VIX_FUT_MERGED.index.get_loc('2008-01-02'),VIX_FUT_MERGED.index.get_loc('2009-12-31')]
CRASH_S_I_gfc = [STRADDLE_MERGED.index.get_loc('2008-01-02'),STRADDLE_MERGED.index.get_loc('2009-12-31')]

CRASH_VS_I_c = [VAR_SWAPS_MERGED.index.get_loc('2020-02-20'),VAR_SWAPS_MERGED.index.get_loc('2020-03-25')]
CRASH_VF_I_c = [VIX_FUT_MERGED.index.get_loc('2020-02-20'),VIX_FUT_MERGED.index.get_loc('2020-03-25')]
CRASH_S_I_c = [STRADDLE_MERGED.index.get_loc('2020-02-20'),STRADDLE_MERGED.index.get_loc('2020-03-25')]

In [70]:
a = pd.concat([scaled_VS[:CRASH_VS_I_gfc[0]],scaled_VS[CRASH_VS_I_gfc[1]:]]) 
b = pd.concat([scaled_VF[:CRASH_VF_I_gfc[0]],scaled_VF[CRASH_VF_I_gfc[1]:]]) 
c = pd.concat([scaled_S[:CRASH_S_I_gfc[0]],scaled_S[CRASH_S_I_gfc[1]:]]) 

scaled_NC_VS = pd.concat([a[:CRASH_VS_I_c[0]],a[CRASH_VS_I_c[1]:]]) 
scaled_NC_VF = pd.concat([b[:CRASH_VF_I_c[0]],b[CRASH_VF_I_c[1]:]]) 
scaled_NC_S = pd.concat([c[:CRASH_S_I_c[0]],c[CRASH_S_I_c[1]:]]) 

In [74]:
a = pd.concat([XS_VAR_SWAPS[:CRASH_VS_I_gfc[0]],XS_VAR_SWAPS[CRASH_VS_I_gfc[1]:]]) 
b = pd.concat([XS_VIX_FUT[:CRASH_VF_I_gfc[0]],XS_VIX_FUT[CRASH_VF_I_gfc[1]:]]) 
c = pd.concat([XS_STRADDLE[:CRASH_S_I_gfc[0]],XS_STRADDLE[CRASH_S_I_gfc[1]:]]) 

XS_NC_VS = pd.concat([a[:CRASH_VS_I_c[0]],a[CRASH_VS_I_c[1]:]]) 
XS_NC_VF = pd.concat([b[:CRASH_VF_I_c[0]],b[CRASH_VF_I_c[1]:]]) 
XS_NC_S = pd.concat([c[:CRASH_S_I_c[0]],c[CRASH_S_I_c[1]:]]) 

### Robustness of Slope as a predictor

### Variance Swap

### Next-month Returns

In [75]:
m_ES_VS = cr_VAR_SWAPS
m_ES_VS['Slope_Quantile'] = pd.qcut(scaled_VS['Slope'], q=5, labels=False)
m_VS_Q_Ret = m_ES_VS.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(m_VS_Q_Ret.min() - m_VS_Q_Ret.max()).T
range_row.index = ['Next-Month Returns']
m_VS_Q_Ret = pd.concat([m_VS_Q_Ret, range_row])

In [76]:
VS_Robustness = VS_Q_Ret.iloc[5:,:]
VS_Robustness = pd.concat([VS_Robustness, m_VS_Q_Ret.iloc[5:,:]])

### Crisis Removed

In [77]:
NC_ES_VS = XS_NC_VS[1:].reset_index(drop=True)
NC_ES_VS['Slope_Quantile'] = pd.qcut(scaled_NC_VS['Slope'], q=5, labels=False)
NC_VS_Q_Ret = NC_ES_VS.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(NC_VS_Q_Ret.min() - NC_VS_Q_Ret.max()).T
range_row.index = ['Crisis Removed']
NC_VS_Q_Ret = pd.concat([NC_VS_Q_Ret, range_row])

In [78]:
VS_Robustness = pd.concat([VS_Robustness, NC_VS_Q_Ret.iloc[5:,:]])

### Bottom 5% Slope removed

In [79]:
ES_VS = XS_VAR_SWAPS[1:].reset_index(drop=True)
ES_VS['Slope'] = scaled_VS['Slope']
ES_VS['Slope_Quantile'] = pd.qcut(scaled_VS['Slope'], q=5, labels=False)
ES_VS = ES_VS[ES_VS['Slope'] > ES_VS['Slope'].quantile(0.05)]
ES_VS = ES_VS.drop('Slope',axis=1)
BFIVE_VS_Q_Ret = ES_VS.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(BFIVE_VS_Q_Ret.min() - BFIVE_VS_Q_Ret.max()).T
range_row.index = ['Bottom 5% Slope removed']
BFIVE_VS_Q_Ret = pd.concat([BFIVE_VS_Q_Ret, range_row])

In [80]:
VS_Robustness = pd.concat([VS_Robustness, BFIVE_VS_Q_Ret.iloc[5:,:]])

### Slope = VIX12 - VIX1

In [81]:
NM_ES_VS = XS_VAR_SWAPS[1:].reset_index(drop=True)
NM_ES_VS['Slope_Quantile'] = pd.qcut((VAR_SWAPS_MERGED['12 month'] - VAR_SWAPS_MERGED['1 month']), q=5, labels=False).reset_index(drop=True)
NM_VS_Q_Ret = NM_ES_VS.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(NM_VS_Q_Ret.min() - NM_VS_Q_Ret.max()).T
range_row.index = ['Slope = VIX 12-VIX 1']
NM_VS_Q_Ret = pd.concat([NM_VS_Q_Ret, range_row])

In [82]:
VS_Robustness = pd.concat([VS_Robustness, NM_VS_Q_Ret.iloc[5:,:]])

In [83]:
VS_Robustness

,V_S_R 1 month,V_S_R 2 month,V_S_R 3 month,V_S_R 6 month,V_S_R 9 month,V_S_R 12 month
Range,-1.417752,-0.664489,-0.637470,-0.509804,-0.431595,-0.388887
Next-Month Returns,-29.190882,-23.138428,-19.845575,-14.135011,-11.330696,-10.046927
Crisis Removed,-1.242815,-0.608508,-0.514003,-0.381530,-0.293454,-0.264376
Bottom 5% Slope removed,-0.996327,-0.643279,-0.637470,-0.509804,-0.431595,-0.388887
Slope = VIX 12-VIX 1,-2.809722,-1.507775,-0.977455,-0.375228,-0.217102,-0.184494


### VIX Futures

### Next-month Returns

In [84]:
m_ES_VF = cr_VIX_FUT
m_ES_VF['Slope_Quantile'] = pd.qcut(scaled_VF['Slope'], q=5, labels=False)
m_VF_Q_Ret = m_ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100

std_errors = m_ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').std(numeric_only=True) / np.sqrt(m_ES_VF.groupby('Slope_Quantile').size())
m_VF_Q_Ret = m_VF_Q_Ret.add_suffix('_Mean')
std_errors = std_errors.add_suffix('_StdErr')
m_VF_Q_Ret = pd.concat([m_VF_Q_Ret, std_errors], axis=1)

range_row = pd.DataFrame(m_VF_Q_Ret.min() - m_VF_Q_Ret.max()).T
range_row.index = ['Next-Month Returns']
m_VF_Q_Ret = pd.concat([m_VF_Q_Ret, range_row])

In [85]:
grouped = m_ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile')
std_dev = grouped.std(numeric_only=True)
counts = grouped.count()
std_error = std_dev / np.sqrt(counts)*100
std_error_range(std_error)

,V_F_R 1 month,V_F_R 2 month,V_F_R 3 month,V_F_R 6 month,V_F_R 9 month,V_F_R 12 month


In [86]:
VF_Robustness = VF_Q_Ret.iloc[5:,:]
VF_Robustness = pd.concat([VF_Robustness, m_VF_Q_Ret.iloc[5:,:]])

NameError: name 'VF_Q_Ret' is not defined

### Crisis Removed

In [87]:
NC_ES_VF = XS_NC_VF[1:].reset_index(drop=True,inplace=False)
NC_ES_VF['Slope_Quantile'] = pd.qcut(scaled_NC_VF['Slope'], q=5, labels=False)
NC_VF_Q_Ret = NC_ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(NC_VF_Q_Ret.min() - NC_VF_Q_Ret.max()).T
range_row.index = ['Crisis Removed']
NC_VF_Q_Ret = pd.concat([NC_VF_Q_Ret, range_row])

In [88]:
grouped = NC_ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile')
std_dev = grouped.std(numeric_only=True)
counts = grouped.count()
std_error = (std_dev / np.sqrt(counts))*100
std_error_range(std_error)

,V_F_R 1 month,V_F_R 2 month,V_F_R 3 month,V_F_R 6 month,V_F_R 9 month,V_F_R 12 month
0,0.228843,0.180372,0.159872,0.151163,0.150127,0.152774


In [89]:
VF_Robustness = pd.concat([VF_Robustness, NC_VF_Q_Ret.iloc[5:,:]])

NameError: name 'VF_Robustness' is not defined

### Bottom 5% Slope removed

In [2561]:
ES_VF = XS_VIX_FUT[1:].reset_index(drop=True,inplace=False)
ES_VF['Slope'] = scaled_VF['Slope']
ES_VF['Slope_Quantile'] = pd.qcut(scaled_VF['Slope'], q=5, labels=False)
ES_VF = ES_VF[ES_VF['Slope'] > ES_VF['Slope'].quantile(0.05)]
ES_VF = ES_VF.drop('Slope',axis=1)
BFIVE_VF_Q_Ret = ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(BFIVE_VF_Q_Ret.min() - BFIVE_VF_Q_Ret.max()).T
range_row.index = ['Bottom 5% Slope removed']
BFIVE_VF_Q_Ret = pd.concat([BFIVE_VF_Q_Ret, range_row])

In [2562]:
grouped = ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile')
std_dev = grouped.std(numeric_only=True)
counts = grouped.count()
std_error = (std_dev / np.sqrt(counts))*100
std_error_range(std_error)

,V_F_R 1 month,V_F_R 2 month,V_F_R 3 month,V_F_R 6 month,V_F_R 9 month,V_F_R 12 month
0,0.210295,0.173555,0.13246,0.114937,0.115222,0.124234


In [2563]:
VF_Robustness = pd.concat([VF_Robustness.iloc[:,2:], BFIVE_VF_Q_Ret.iloc[5:,:]])

### Slope = VIX12 - VIX1

In [2564]:
NM_ES_VF = XS_VIX_FUT[1:].reset_index(drop=True)
NM_ES_VF['Slope_Quantile'] = pd.qcut((VIX_FUT_MERGED['12 month'] - VIX_FUT_MERGED['1 month']), q=5, labels=False).reset_index(drop=True)
NM_VF_Q_Ret = NM_ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(NM_VF_Q_Ret.min() - NM_VF_Q_Ret.max()).T
range_row.index = ['Slope = VIX 12-VIX 1']
NM_VF_Q_Ret = pd.concat([NM_VF_Q_Ret, range_row])

In [2565]:
grouped = NM_ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile')
std_dev = grouped.std(numeric_only=True)
counts = grouped.count()
std_error = (std_dev / np.sqrt(counts))*100
std_error_range(std_error)

,V_F_R 1 month,V_F_R 2 month,V_F_R 3 month,V_F_R 6 month,V_F_R 9 month,V_F_R 12 month
0,0.226377,0.167923,0.138413,0.121738,0.110395,0.102471


In [2566]:
VF_Robustness = pd.concat([VF_Robustness, NM_VF_Q_Ret.iloc[5:,:]])

In [2567]:
VF_Robustness

,V_F_R 1 month,V_F_R 2 month,V_F_R 3 month,V_F_R 6 month,V_F_R 9 month,V_F_R 12 month,V_F_R 1 month_Mean,V_F_R 2 month_Mean,V_F_R 3 month_Mean,V_F_R 6 month_Mean,V_F_R 9 month_Mean,V_F_R 12 month_Mean,V_F_R 1 month_StdErr,V_F_R 2 month_StdErr,V_F_R 3 month_StdErr,V_F_R 6 month_StdErr,V_F_R 9 month_StdErr,V_F_R 12 month_StdErr
Range,-0.272033,-0.210087,-0.187775,-0.149724,-0.133765,-0.124185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Crisis Removed,-0.300214,-0.249844,-0.237720,-0.208887,-0.206551,-0.195201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bottom 5% Slope removed,-0.272033,-0.258889,-0.216900,-0.179595,-0.183399,-0.179653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Slope = VIX 12-VIX 1,-0.641890,-0.336317,-0.175945,-0.155629,-0.151765,-0.164948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Straddle

### Next-month Returns

In [2568]:
m_ES_S = cr_STRADDLE
m_ES_S['Slope_Quantile'] = pd.qcut(scaled_S['Slope'], q=5, labels=False)
m_S_Q_Ret = m_ES_S.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(m_S_Q_Ret.min() - m_S_Q_Ret.max()).T
range_row.index = ['Next-Month Returns']
m_S_Q_Ret = pd.concat([m_S_Q_Ret, range_row])

In [2569]:
grouped = m_ES_S.sort_values(by='Slope_Quantile').groupby('Slope_Quantile')
std_dev = grouped.std(numeric_only=True)
counts = grouped.count()
std_error = (std_dev / np.sqrt(counts))*100
std_error_range(std_error)

,DATE,S_R 1 month,S_R 12 month,S_R 2 month,S_R 3 month,S_R 6 month,S_R 9 month
0,NaN,1.32931,0.378021,0.852737,0.66886,0.497228,0.400457


In [2570]:
S_Robustness = S_Q_Ret.iloc[5:,:]
S_Robustness = pd.concat([S_Robustness, m_S_Q_Ret.iloc[5:,:]])

### Crisis Removed

In [2571]:
NC_ES_S = XS_NC_S[1:].reset_index(drop=True)
NC_ES_S['Slope_Quantile'] = pd.qcut(scaled_NC_S['Slope'], q=5, labels=False)
NC_S_Q_Ret = NC_ES_S.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(NC_S_Q_Ret.min() - NC_S_Q_Ret.max()).T
range_row.index = ['Crisis Removed']
NC_S_Q_Ret = pd.concat([NC_S_Q_Ret, range_row])

In [2572]:
grouped = NC_ES_S.sort_values(by='Slope_Quantile').groupby('Slope_Quantile')
std_dev = grouped.std(numeric_only=True)
counts = grouped.count()
std_error = (std_dev / np.sqrt(counts))*100
std_error_range(std_error)

,S_R 1 month,S_R 2 month,S_R 3 month,S_R 6 month,S_R 9 month,S_R 12 month
0,0.246494,0.145025,0.117304,0.080505,0.067215,0.064877


In [2573]:
S_Robustness = pd.concat([S_Robustness, NC_S_Q_Ret.iloc[5:,:]])

### Bottom 5% Slope removed

In [2574]:
ES_S = XS_STRADDLE[1:].reset_index(drop=True)
ES_S['Slope'] = scaled_S['Slope']
ES_S['Slope_Quantile'] = pd.qcut(scaled_S['Slope'], q=5, labels=False)
ES_S = ES_S[ES_S['Slope'] > ES_S['Slope'].quantile(0.05)]
ES_S = ES_S.drop('Slope',axis=1)
BFIVE_S_Q_Ret = ES_S.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(BFIVE_S_Q_Ret.min() - BFIVE_S_Q_Ret.max()).T
range_row.index = ['Bottom 5% Slope removed']
BFIVE_S_Q_Ret = pd.concat([BFIVE_S_Q_Ret, range_row])

In [2575]:
grouped = ES_S.sort_values(by='Slope_Quantile').groupby('Slope_Quantile')
std_dev = grouped.std(numeric_only=True)
counts = grouped.count()
std_error = (std_dev / np.sqrt(counts))*100
std_error_range(std_error)

,S_R 1 month,S_R 2 month,S_R 3 month,S_R 6 month,S_R 9 month,S_R 12 month
0,0.259398,0.154387,0.123857,0.084065,0.068292,0.064425


In [2576]:
S_Robustness = pd.concat([S_Robustness, BFIVE_S_Q_Ret.iloc[5:,:]])

### Slope = VIX12 - VIX1

In [2577]:
NM_ES_S = XS_STRADDLE[1:].reset_index(drop=True)
NM_ES_S['Slope_Quantile'] = pd.qcut((STRADDLE_MERGED['12 month'] - STRADDLE_MERGED['1 month']), q=5, labels=False).reset_index(drop=True)
NM_S_Q_Ret = NM_ES_S.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(NM_S_Q_Ret.min() - NM_S_Q_Ret.max()).T
range_row.index = ['Slope = VIX 12-VIX 1']
NM_S_Q_Ret = pd.concat([NM_S_Q_Ret, range_row])

In [2578]:
grouped = NM_ES_S.sort_values(by='Slope_Quantile').groupby('Slope_Quantile')
std_dev = grouped.std(numeric_only=True)
counts = grouped.count()
std_error = (std_dev / np.sqrt(counts))*100
std_error_range(std_error)

,S_R 1 month,S_R 2 month,S_R 3 month,S_R 6 month,S_R 9 month,S_R 12 month
0,0.247951,0.158578,0.128555,0.093329,0.075638,0.072004


In [2579]:
S_Robustness = pd.concat([S_Robustness.iloc[:,1:], NM_S_Q_Ret.iloc[5:,:]])

In [2580]:
S_Robustness

,S_R 1 month,S_R 2 month,S_R 3 month,S_R 6 month,S_R 9 month,S_R 12 month
Range,-0.671157,-0.570107,-0.530845,-0.369153,-0.310669,-0.282248
Next-Month Returns,-16.016065,-12.201679,-10.866073,-7.892658,-6.729386,-6.514062
Crisis Removed,-0.487437,-0.370976,-0.389452,-0.311105,-0.276049,-0.257258
Bottom 5% Slope removed,-0.617242,-0.417197,-0.380627,-0.254073,-0.218522,-0.195672
Slope = VIX 12-VIX 1,-0.480404,-0.355143,-0.307407,-0.255006,-0.216822,-0.205750


### Crash indicator

### Variance Swap

In [2581]:
# Set the window size and quantile level
window_size = 21
quantile_level = 0.01

# Create a new DataFrame to store the indicators
CRASH_VS = pd.DataFrame()

# Loop through each column
for col in XS_VAR_SWAPS.iloc[:,1:].columns:
    # Calculate the rolling quantile for the current column
    rolling_quantile = XS_VAR_SWAPS[col].rolling(window=window_size).quantile(quantile_level)
    
    # Create an indicator column for the current column
    indicator_col = (XS_VAR_SWAPS[col] <= rolling_quantile).astype(int)
    
    # Rename the indicator column
    indicator_col.name = col + '_indicator'
    
    # Append the indicator column to the indicators DataFrame
    CRASH_VS = pd.concat([CRASH_VS, indicator_col], axis=1)

### Vix Future

In [2582]:
# Set the window size and quantile level
window_size = 21
quantile_level = 0.01

# Create a new DataFrame to store the indicators
CRASH_VF = pd.DataFrame()

# Loop through each column
for col in XS_VIX_FUT.iloc[:,:].columns:
    # Calculate the rolling quantile for the current column
    rolling_quantile = XS_VIX_FUT[col].rolling(window=window_size).quantile(quantile_level)
    
    # Create an indicator column for the current column
    indicator_col = (XS_VIX_FUT[col] <= rolling_quantile).astype(int)
    
    # Rename the indicator column
    indicator_col.name = col + '_indicator'
    
    # Append the indicator column to the indicators DataFrame
    CRASH_VF = pd.concat([CRASH_VF, indicator_col], axis=1)

### Straddle

In [2583]:
# Set the window size and quantile level
window_size = 21
quantile_level = 0.01

# Create a new DataFrame to store the indicators
CRASH_S= pd.DataFrame()

# Loop through each column
for col in XS_STRADDLE.iloc[:,:].columns:
    # Calculate the rolling quantile for the current column
    rolling_quantile = XS_STRADDLE[col].rolling(window=window_size).quantile(quantile_level)
    
    # Create an indicator column for the current column
    indicator_col = (XS_STRADDLE[col] <= rolling_quantile).astype(int)
    
    # Rename the indicator column
    indicator_col.name = col + '_indicator'
    
    # Append the indicator column to the indicators DataFrame
    CRASH_S = pd.concat([CRASH_S, indicator_col], axis=1)

### Crash

In [90]:
Crash = pd.read_csv(DATA_PATH + '/CRASH.csv')
Crash = Crash.iloc[:,1:]
Crash = Crash.rename({'date':'DATE'},axis=1)
Crash.set_index('DATE', inplace=True)
Crash.index = pd.to_datetime(Crash.index)

### Variance Risk Premium

In [91]:
VRP = pd.read_csv(DATA_PATH + '/VRP_data.csv')
VRP = VRP.iloc[:,1:]
VRP = VRP.rename({'date':'DATE'},axis=1)
VRP.set_index('DATE', inplace=True)
VRP.index = pd.to_datetime(VRP.index)
VRP=VRP*-1

In [92]:
STRADDLE_VRP = pd.merge(STRADDLE_MERGED,VRP, how='inner', left_index=True, right_index=True)
VF_VRP = pd.merge(VIX_FUT_MERGED,VRP, how='inner', left_index=True, right_index=True)

### Dealer Leverage

In [93]:
DEALER_LEVERAGE = pd.read_csv(DATA_PATH + '/BROKER_DEALER.csv')
DEALER_LEVERAGE = DEALER_LEVERAGE.iloc[:,1:]
DEALER_LEVERAGE = DEALER_LEVERAGE.rename({'Date':'DATE'},axis=1)
DEALER_LEVERAGE.set_index('DATE',inplace=True)
DEALER_LEVERAGE.index = pd.to_datetime(DEALER_LEVERAGE.index)

In [94]:
DEALER_LEVERAGE.index = pd.PeriodIndex(DEALER_LEVERAGE.index, freq='Q')
DEALER_LEVERAGE = DEALER_LEVERAGE.shift(1, freq='Q')

In [95]:
daily_data = []
for quarter, value in DEALER_LEVERAGE.itertuples():
    quarter_start = quarter.start_time
    quarter_end = quarter.end_time
    days = pd.date_range(start=quarter_start, end=quarter_end, freq='D')
    daily_values = pd.DataFrame({'D_L': [value] * len(days)}, index=days)
    daily_data.append(daily_values)

# Concatenate all daily data
DEALER_LEVERAGE = pd.concat(daily_data)

### Merging all straddle and futures returns and indicators

In [115]:
scaled_S['DATE'] = STRADDLE_MERGED.index
scaled_S.set_index('DATE',inplace=True)
scaled_VF['DATE'] = VIX_FUT_MERGED.index
scaled_VF.set_index('DATE',inplace=True)
STRADDLE_MERGE = STRADDLE_MERGED.drop(columns=['S_R 1 month','S_R 2 month','S_R 3 month','S_R 6 month','S_R 9 month','S_R 12 month'])
VIX_FUT_MERGE = VIX_FUT_MERGED.drop(columns=['V_F_R 1 month','V_F_R 2 month','V_F_R 3 month','V_F_R 6 month','V_F_R 9 month','V_F_R 12 month'])

In [116]:
STRADDLE_REG = pd.merge(STRADDLE_MERGE, XS_STRADDLE, how='inner', left_index=True, right_index=True)
STRADDLE_REG = pd.merge(STRADDLE_REG,VRP, how='inner', left_index=True, right_index=True)
STRADDLE_REG = pd.merge(STRADDLE_REG,DEALER_LEVERAGE, how='inner', left_index=True, right_index=True)
STRADDLE_REG = pd.merge(STRADDLE_REG,Crash, how='inner', left_index=True, right_index=True)
STRADDLE_REG = STRADDLE_REG.drop('CRASH',axis=1)
scaled = scaler.fit_transform(STRADDLE_REG)
STRADDLE_REG_S = pd.DataFrame(data=scaled,columns=STRADDLE_REG.columns,index=STRADDLE_REG.index)
STRADDLE_REG_S = pd.merge(STRADDLE_REG_S,Crash, how='inner', left_index=True, right_index=True)
STRADDLE_REG_S = pd.merge(STRADDLE_REG_S,scaled_S['Slope'], how='inner', left_index=True, right_index=True)

In [117]:
VIX_FUT_REG = pd.merge(VIX_FUT_MERGE,XS_VIX_FUT, how='inner', left_index=True, right_index=True)
VIX_FUT_REG = pd.merge(VIX_FUT_REG,VRP, how='inner', left_index=True, right_index=True)
VIX_FUT_REG = pd.merge(VIX_FUT_REG,DEALER_LEVERAGE, how='inner', left_index=True, right_index=True)
VIX_FUT_REG = pd.merge(VIX_FUT_REG,Crash, how='inner', left_index=True, right_index=True)
VIX_FUT_REG = VIX_FUT_REG.drop('CRASH',axis=1)

In [118]:
scaled = scaler.fit_transform(VIX_FUT_REG)
VIX_FUT_REG_S = pd.DataFrame(data=scaled,columns=VIX_FUT_REG.columns,index=VIX_FUT_REG.index)
VIX_FUT_REG_S = pd.merge(VIX_FUT_REG_S,Crash, how='inner', left_index=True, right_index=True)
VIX_FUT_REG_S = pd.merge(VIX_FUT_REG_S,scaled_VF['Slope'], how='inner', left_index=True, right_index=True)

### Slope as an Incremental Predictor of Variance Asset Returns

### Straddle

In [120]:
Y = (STRADDLE_REG['S_R 12 month'][1:]*10000).reset_index(drop=True)

In [121]:
SLOPE = STRADDLE_REG_S['Slope'][:-1]
r = STRADDLE_REG_S['S_R 12 month'][:-1]
Crash = STRADDLE_REG_S['CRASH'][:-1]
VIX = STRADDLE_REG_S['1 month'][:-1]
VRP = STRADDLE_REG_S['VRP'][:-1]
DL = STRADDLE_REG_S['D_L'][:-1]

X = pd.concat([SLOPE,r,Crash,VIX,VRP,DL],axis=1)
X = X.reset_index(drop=True)

In [122]:
model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':0})
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           S_R 12 month   R-squared (uncentered):                   0.008
Model:                            OLS   Adj. R-squared (uncentered):              0.007
Method:                 Least Squares   F-statistic:                              4.168
Date:                Sat, 02 Dec 2023   Prob (F-statistic):                    0.000346
Time:                        15:24:35   Log-Likelihood:                         -40367.
No. Observations:                6195   AIC:                                  8.075e+04
Df Residuals:                    6189   BIC:                                  8.079e+04
Df Model:                           6                                                  
Covariance Type:                  HAC                                                  
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Slope          -11.6483      4.120     -2.828      0.005     -19.723      -3.574
S_R 12 month     4.8783      5.142      0.949      0.343      -5.199      14.956
CRASH           20.4714     10.332      1.981      0.048       0.222      40.721
1 month         17.5008     38.183      0.458      0.647     -57.337      92.338
VRP             22.2142     39.374      0.564      0.573     -54.958      99.386
D_L             -2.2296      2.649     -0.842      0.400      -7.422       2.962
==============================================================================
Omnibus:                     3211.286   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           103199.158
Skew:                           1.876   Prob(JB):                         0.00
Kurtosis:                      22.640   Cond. No.                         17.6
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 0 lags and without small sample correction
"""

### VIX Futures

In [220]:
Y = (VIX_FUT_REG['V_F_R 12 month'][1:]*10000).reset_index(drop=True)

In [221]:
SLOPE = VIX_FUT_REG_S['Slope'][:-1]
r = VIX_FUT_REG_S['V_F_R 12 month'][:-1]
Crash = VIX_FUT_REG_S['CRASH'][:-1]
VIX = VIX_FUT_REG_S['1 month'][:-1]
VRP = VIX_FUT_REG_S['VRP'][:-1]
DL = VIX_FUT_REG_S['D_L'][:-1]

X = pd.concat([SLOPE,r,Crash,VIX,VRP,DL],axis=1)
X = X.reset_index(drop=True)

In [222]:
model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':0})
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:         V_F_R 12 month   R-squared (uncentered):                   0.021
Model:                            OLS   Adj. R-squared (uncentered):              0.019
Method:                 Least Squares   F-statistic:                              2.300
Date:                Sat, 02 Dec 2023   Prob (F-statistic):                      0.0321
Time:                        15:45:16   Log-Likelihood:                         -30440.
No. Observations:                4351   AIC:                                  6.089e+04
Df Residuals:                    4345   BIC:                                  6.093e+04
Df Model:                           6                                                  
Covariance Type:                  HAC                                                  
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Slope            -10.1886      6.557     -1.554      0.120     -23.039       2.662
V_F_R 12 month   -37.3809     12.774     -2.926      0.003     -62.417     -12.345
CRASH            -12.0149     33.893     -0.354      0.723     -78.444      54.414
1 month           31.9376     49.600      0.644      0.520     -65.277     129.152
VRP               26.8260     50.874      0.527      0.598     -72.886     126.538
D_L               -4.9993      4.635     -1.079      0.281     -14.084       4.085
==============================================================================
Omnibus:                     6372.983   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         18776471.948
Skew:                          -7.931   Prob(JB):                         0.00
Kurtosis:                     324.433   Cond. No.                         17.7
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 0 lags and without small sample correction
"""